In [75]:
# client.py
import pyarrow as pa
import pyarrow.flight

class FlightClient:
    def __init__(self, server_location):
        print('    debug 0:__inti__(): initialize client-to-server connection')
        self.client = pa.flight.connect(location=server_location)

    def upload_dataset(self, table, dataset_path):
        print('    debug 1:upload_dataset(): upload table to server and save to parquet')
        upload_descriptor = pa.flight.FlightDescriptor.for_path(dataset_path)
        writer, flight = self.client.do_put(upload_descriptor, table.schema)
        writer.write_table(table)
        writer.close()
        return flight

    def show_metadata(self, flight):
        print('    debug 2:retrieve_metadata(): show metadata for a flight object')
        descriptor = flight.descriptor
        print("\tpath:", descriptor.path[0].decode(), "rows:", flight.total_records, "size:", flight.total_bytes)
        schema_str = flight.schema.to_string()
        schema_str = '\t'*2 + schema_str.replace('\n', '\n\t\t')
        print(schema_str)

    def list_datasets(self):
        print('    debug 3:list_datasets(): list all datasets on the server')
        for flight in self.client.list_flights():
            self.show_metadata(flight)
            print("")
            
    def print_metadata(self,dataset_path):
        flight = self.client.get_flight_info(pa.flight.FlightDescriptor.for_path(dataset_path))
        self.show_metadata(flight)
        
    def print_dataset(self, reader):
        print('    debug 4:read_and_print_dataset(): read dataframe from the server and print head of the table')
        read_table = reader.read_all()
        print(read_table.to_pandas().head())

    def drop_dataset(self, dataset):
        print('    debug:drop_dataset(): drop dataset')
        self.client.do_action(pa.flight.Action("drop_dataset", dataset.encode()))

if __name__ == "__main__":
    print('# 1.Create client-to-server connection')
    c = FlightClient("grpc://0.0.0.0:8816")
    
    print('# 2.Upload example dataframe to parquet')
    data_table = pa.table([["Alex", "Bob", "Maya"],[7,8,9]], names=["character","age"])
    c.upload_dataset(data_table, "uploaded.parquet")
    
    print('# 3.Check the result from upload')
    c.print_metadata('uploaded.parquet')
    c.list_datasets()

    print('# 4.Browse datasets on servers')
    read_flights = list(c.client.list_flights())
    read_flight = read_flights[0]
    read_reader = c.client.do_get(read_flight.endpoints[0].ticket)
    c.print_dataset(read_reader)

    print('# 5. delete the dataset')
    c.drop_dataset("uploaded.parquet")
    c.list_datasets()

# 1.Create client-to-server connection
    debug 0:__inti__(): initialize client-to-server connection
# 2.Upload example dataframe to parquet
    debug 1:upload_dataset(): upload table to server and save to parquet
# 3.Check the result from upload
    debug 2:retrieve_metadata(): show metadata for a flight object
	path: uploaded.parquet rows: 3 size: 545
		character: string
		age: int64
    debug 3:list_datasets(): list all datasets on the server
    debug 2:retrieve_metadata(): show metadata for a flight object
	path: uploaded.parquet rows: 3 size: 545
		character: string
		age: int64

# 4.Browse datasets on servers
    debug 4:read_and_print_dataset(): read dataframe from the server and print head of the table
  character  age
0      Alex    7
1       Bob    8
2      Maya    9
# 5. delete the dataset
    debug:drop_dataset(): drop dataset
    debug 3:list_datasets(): list all datasets on the server
